### Import Library

In [31]:
import pandas as pd
import validators

#### Mengelola Nilai yang Hilang

In [11]:
def handle_missing_values(data):
    #identifikasi nilai yang hilang
    missing_values = data[['area', 'floor']].isnull().sum()
    print("Jumlah nilai yang hilang sebelum imputasi:")
    print(missing_values)

    #imputasi nilai yang hilang

    #data pada kolom area diubah menjadi float dan nilai yang hilang diganti dengan median
    data['area'] = data['area'].str.replace(' m²', '').astype(float)
    median_area = data['area'].median()
    data['area'] = data['area'].fillna(median_area)

    #pakai angka pertama sebagai lantai dan data diubah menjadi float
    data['floor'] = data['floor'].apply(lambda x: float(x.split('/')[0].strip()) if isinstance(x, str) else x)
    #kolom floor, nilai yang hilang diganti dengan nilai median
    median_floor = data['floor'].median()
    data['floor'] = data['floor'].fillna(median_floor)

    #verifikasi imputasi berhasil atau tidak
    missing_values_after = data[['area', 'floor']].isnull().sum()
    print("\nJumlah nilai yang hilang setelah imputasi:")
    print(missing_values_after)

    return data

if __name__ == "__main__":
    data = pd.read_csv('house_listings.csv')

    data = handle_missing_values(data)

    #simpan data yang telah diimputasi ke file CSV
    data.to_csv("house_listings_imputed.csv", index=False)

    print("\nData setelah nilai yang hilang diimputasi:")
    print(data.head(25))


Jumlah nilai yang hilang sebelum imputasi:
area     37
floor    37
dtype: int64

Jumlah nilai yang hilang setelah imputasi:
area     0
floor    0
dtype: int64

Data setelah nilai yang hilang diimputasi:
        category    price currency     price_1m2  \
0   Köhnə tikili  137 000      AZN  2 630 AZN/m²   
1    Yeni tikili  183 000      AZN  1 790 AZN/m²   
2   Köhnə tikili  145 000      AZN  2 230 AZN/m²   
3   Köhnə tikili  190 000      AZN  2 000 AZN/m²   
4    Yeni tikili  294 000      AZN  1 550 AZN/m²   
5   Köhnə tikili   87 000      AZN  1 930 AZN/m²   
6    Yeni tikili  550 000      AZN  2 530 AZN/m²   
7    Yeni tikili  125 000      AZN  2 160 AZN/m²   
8   Köhnə tikili  150 000      AZN  2 730 AZN/m²   
9    Yeni tikili   62 000      AZN  1 510 AZN/m²   
10   Yeni tikili  375 000      AZN  2 270 AZN/m²   
11   Yeni tikili  729 000      AZN  3 390 AZN/m²   
12  Köhnə tikili  365 000      AZN  2 090 AZN/m²   
13   Yeni tikili  105 000      AZN  1 240 AZN/m²   
14   Yeni tikili 

#### Mengatasi Outlier

In [17]:
def outliers(data):
    #bersihkan data pada kolom price dan diubah menjadi float
    data['price'] = data['price'].str.replace(',', '').str.replace(' ', '')
    data['price'] = data['price'].astype(float)

    #hitung IQR (Interquartile Range)
    Q1 = data['price'].quantile(0.25)
    Q3 = data['price'].quantile(0.75)
    IQR = Q3 - Q1

    #batas bawah dan batas atas untuk outlier
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outlier_indices = (data['price'] < lower_bound) | (data['price'] > upper_bound)
    #ganti outlier dengan nilai median
    data.loc[outlier_indices, 'price'] = data['price'].median()  

    return data

if __name__ == "__main__":
    data = pd.read_csv('house_listings.csv')

    data = outliers(data)

    #simpan data yang telah diubah ke file CSV
    data.to_csv("house_listings_no_outliers.csv", index=False)

    print("Data tanpa outlier:")
    print(data.head())


Data tanpa outlier:
       category     price currency     price_1m2  \
0  Köhnə tikili  137000.0      AZN  2 630 AZN/m²   
1   Yeni tikili  183000.0      AZN  1 790 AZN/m²   
2  Köhnə tikili  145000.0      AZN  2 230 AZN/m²   
3  Köhnə tikili  190000.0      AZN  2 000 AZN/m²   
4   Yeni tikili  294000.0      AZN  1 550 AZN/m²   

                                               title  \
0  Satılır 3 otaqlı köhnə tikili 52 m², 8-ci kilo...   
1  Satılır 2 otaqlı yeni tikili 102 m², Neftçilər m.   
2  Satılır 2 otaqlı köhnə tikili 65 m², Nərimanov r.   
3    Satılır 3 otaqlı köhnə tikili 95 m², Gənclik m.   
4     Satılır 3 otaqlı yeni tikili 190 m², Nəsimi r.   

                    address   floor    area title_deed  repair mortgage  \
0  Elşən Süleymanov küç 137   7 / 9   52 m²        var     var      NaN   
1      Mehdi Abbasov küçəsi  2 / 17  102 m²     yoxdur     var      NaN   
2         Atatürk Prospekti   6 / 9   65 m²        var     var      var   
3               Atatürk pr.   

In [ ]:
data = pd.read_csv('house_listings.csv')

#address verif
#cek missing values
if data['address'].isnull().sum() > 0:
    print("WARNING: There are missing values in the 'address' column.")
    #hapus missing addresses
    data = data.dropna(subset=['address'], inplace=False)

#cek apakah ada duplikat data address
if data['address'].duplicated().sum() > 0:
    print("WARNING: There are duplicate addresses in the 'address' column.")
    #hapus data duplikat pada address
    data = data.drop_duplicates(subset=['address'], inplace=False)

#URL validation
#cek missing values
if data['url'].isnull().sum() > 0:
    print("WARNING: There are missing values in the 'url' column.")

#cek format valid URL 
if data['url'].notna().any():  #cek apakah ada URL yang tidak hilang
    #validasi untuk URL yang tidak hilang
    url_pattern = r"https?://[^\s]+?"  #URL pattern
    valid_urls = data['url'].str.contains(url_pattern)
    if not valid_urls.all():
        print("WARNING: There are invalid URLs in the 'url' column.")

data.to_csv('house_listings_baru.csv', index=False)
